In [1]:
#using Revise
using DrWatson
@quickactivate "ABoxWorld"
include(srcdir("ABoxWorld.jl"));

┌ Info: ABoxWorld project environment is loaded and active
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\src\ABoxWorld.jl:10


In [2]:
#conditions.check(NS_Conditions(), Random_NS_Mixture_CHSH())
conditions.check(Original_IC_Bound(), Random_NS_Mixture_CHSH(;non_local_bias=true), :Q)
#sdp_conditions.is_in_NPA(nsboxes.reconstructFullJoint(Random_NS_Mixture_CHSH()))

true

In [3]:
init_box = RandomNSBox((2,2,2,2))
println(init_box.marginals_vec_A)
println(init_box.marginals_vec_B)
println(init_box.joints_mat)
#println(init_box)
#println([init_box.joints_mat[end,end] init_box.marginals_vec_A[end]-init_box.joints_mat[end,end];
#    init_box.marginals_vec_B[end]-init_box.joints_mat[end,end] (1.0-init_box.joints_mat[end,end]-(init_box.marginals_vec_A[end]-init_box.joints_mat[end,end])-(init_box.marginals_vec_B[end]-init_box.joints_mat[end,end])) ])
#fullNSjoint = nsboxes.reconstructFullJoint(init_box)
#fullNSjoint
#println(conditions.check(NS_Conditions(), fullNSjoint))

(normalization_min, normalization_max, marginal_A_limit, marginal_B_limit) = (0.21748000066851247, 1.0, 0.7340213831800354, 0.48345861748847707)
(normalization_min, normalization_max, marginal_A_limit, marginal_B_limit) = (0.43345336072984697, 1.0, 0.592463874345352, 0.8409894863844949)
(normalization_min, normalization_max, marginal_A_limit, marginal_B_limit) = (0.5750108695645304, 1.0, 0.7340213831800354, 0.8409894863844949)
(normalization_min, normalization_max, marginal_A_limit, marginal_B_limit) = (0.07592249183382904, 1.0, 0.592463874345352, 0.48345861748847707)
[0.7340213831800354, 0.592463874345352]
[0.48345861748847707, 0.8409894863844949]
[0.3389119181818678 0.5972152359738263; 0.2974668061614932 0.4912520944560043]


In [4]:


using LinearAlgebra
using TensorOperations
using Parameters
using CairoMakie
CairoMakie.activate!()
#using GLMakie
#GLMakie.activate!()


In [5]:

convert_matrixbox_to_nsjoint = wirings.convert_matrixbox_to_nsjoint
convert_nsjoint_to_matrixbox = wirings.convert_nsjoint_to_matrixbox


# Boxes as 2x2x2x2 tensors
MaxMixedBox = nsboxes.reconstructFullJoint(UniformRandomBox((2, 2, 2, 2)))
PR(μ, ν, σ) = nsboxes.reconstructFullJoint(PRBoxesCHSH(;μ=μ, ν=ν, σ=σ))
CanonicalPR = PR(0, 0, 0)
PL(α, γ, β, λ) = nsboxes.reconstructFullJoint(LocalDeterministicBoxesCHSH(;α=α, γ=γ, β=β, λ=λ))
SR = (PL(0,0,0,0) .+ PL(0,1,0,1)) ./ 2
#SR = matrix_to_tensor(non_local_boxes.utils.SR)
#PRprime = matrix_to_tensor(non_local_boxes.utils.PRprime)
#P0 = matrix_to_tensor(non_local_boxes.utils.P_0)
#P1 = matrix_to_tensor(non_local_boxes.utils.P_1)


CHSH_score = games.canonical_CHSH_score
#CHSHprime_score = games.CHSH_score_generator(-1,1,1,1; batched=false)
CHSHprime_score = games.CHSH_score_generator(1,-1,1,1; batched=false)


BoxProduct = wirings.tensorized_boxproduct
reduc_BoxProduct = wirings.reduc_tensorized_boxproduct
BoxProduct_matrices(w::Matrix{<:Real}, matrixbox1::Matrix{Float64}, matrixbox2::Matrix{Float64}) = convert_nsjoint_to_matrixbox(reduc_BoxProduct(w, matrixbox1, matrixbox2))


IC_Bound = Original_IC_Bound()
IC_Bound_LHS(P::Array{Float64, 4}) = conditions.evaluate(IC_Bound, P)
IC_Bound_LHS(W::Matrix{<:Real}, P_mat::Matrix{Float64}, Q_mat::Matrix{Float64}) = games.Original_IC_Bound_score(W, P_mat, Q_mat)
IC_Bound_LHS(W::Vector{<:Real}, P_mat::Matrix{Float64}, Q_mat::Matrix{Float64}) = games.Original_IC_Bound_score(W, P_mat, Q_mat)

IC_MutInfo = games.MutInfo_IC_vanDam_score

function is_in_IC(P::Array{Float64, 4})
    return conditions.check(IC_Bound, P, :Q)
end


is_in_IC (generic function with 1 method)

In [6]:
include(scriptsdir("WiringsSearch", "WiringsRandSearch.jl"))

greedy_lifting_extremal_wiring_search (generic function with 1 method)

In [12]:

c_config = WiringsRandomSearchConfig(mode=:uniform, 
                                    approach_from=:above, 
                                    search_precision=4e-3, 
                                    precision=1.4e-2, 
                                    max_wiring_order=2
                                )
    
data_output = Wirings_Random_Search(c_config; verbose=true)

#data_output, data_filename = produce_or_load(Wirings_Random_Search, 
#                                            c_config,
#                                            mkpath(datadir("WiringsRandomSearch"));
#                                            verbose=true,
#                                            )


┌ Info: 2024-03-19 13:49:13 | message = Searching for IC-violating wirings... | maxrss = 1813.31 MiB
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\scripts\WiringsSearch\wirings_search.jl:30
┌ Warning: Noticed strange movement, non-IC -> IC -> non-IC, only by adding uniform noise; When should this happen?; For safety, we'll just search for something else
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\notebooks\WiringsSearch\WiringsRandSearch.ipynb:54


(:LD, Dict(:α => 1, :γ => 1, :λ => 0, :β => 0))
(:LD, Dict(:α => 0, :γ => 0, :λ => 0, :β => 0))
(:PR, Dict(:ν => 0, :μ => 1, :σ => 0))
with coefficients: 0.217 0.305 0.477


┌ Warning: Noticed strange movement, non-IC -> IC -> non-IC, only by adding uniform noise; When should this happen?; For safety, we'll just search for something else
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\notebooks\WiringsSearch\WiringsRandSearch.ipynb:54
┌ Info: 2024-03-19 13:49:13 | message = Found 1-th box in IC-Q gap in trail 3 of 4 | maxrss = 1813.31 MiB
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\scripts\WiringsSearch\wirings_search.jl:30
┌ Info: The decomposition of the box is:
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\notebooks\WiringsSearch\WiringsRandSearch.ipynb:69
┌ Info: 2024-03-19 13:49:14 | message = Starting search for IC-violating wirings for this box in the IC-Q gap ... | maxrss = 1813.31 MiB
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\scripts\WiringsSearch\wirings_search.jl:30


(:LD, Dict(:α => 1, :γ => 0, :λ => 1, :β => 1))
(:PR, Dict(:ν => 0, :μ => 1, :σ => 1))
(:PR, Dict(:ν => 1, :μ => 0, :σ => 1))
with coefficients: 0.416 0.43 0.154


┌ Info: 2024-03-19 13:49:18 | message = *No* IC-violating wirings found for 1-th box in IC-Q gap in trail 3 of 4 | maxrss = 1813.31 MiB
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\scripts\WiringsSearch\wirings_search.jl:30
┌ Info: 2024-03-19 13:49:18 | message = Found 2-th box in IC-Q gap in trail 4 of 4 | maxrss = 1813.31 MiB
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\scripts\WiringsSearch\wirings_search.jl:30
┌ Info: The decomposition of the box is:
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\notebooks\WiringsSearch\WiringsRandSearch.ipynb:69
┌ Info: 2024-03-19 13:49:18 | message = Starting search for IC-violating wirings for this box in the IC-Q gap ... | maxrss = 1813.31 MiB
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\scripts\WiringsSearch\wirings_search.jl:30
┌ Info: 2024-03-19 13:49:22 | message = *No* IC-violating wirings found for 2-th box in IC-Q gap in trail 4 of 4 | maxrss = 1813.31 MiB
└ @ Main s:\Sync

Dict{String, Any} with 6 entries:
  "approach_from"        => :above
  "IC_violating_wirings" => Any[]
  "mode"                 => :uniform
  "search_precision"     => 0.004
  "max_wiring_order"     => 2
  "precision"            => 0.014